## Zillow 2017 predictions report:

Got an email from the Zillow DS Team:

<i>"We want to be able to predict the property tax assessed values ('taxvaluedollarcnt') of Single Family Properties that had a transaction during 2017.

We have a model already, but we are hoping your insights can help us improve it. I need recommendations on a way to make a better model. Maybe you will create a new feature out of existing ones that works better, try a non-linear regression algorithm, or try to create a different model for each county. Whatever you find that works (or doesn't work) will be useful. Given you have just joined our team, we are excited to see your outside perspective.

One last thing, Zach lost the email that told us where these properties were located. Ugh, Zach :-/. Because property taxes are assessed at the county level, we would like to know what states and counties these are located in."</i>

-- The Zillow Data Science Team

___________________________________________________________________________________________________________________

## My Goal: to predict the values of single unit properties using the zillow database obervations from 2017.




### My Deliverables:

- Constructed ML Regression model that predicts propery tax assessed values (and can be used for our marketing and tech departments).

- Finding the key drivers of property value for single family properties. 


### My Questions: 

- Why are some properties with similar features valued so differently? 

- Do count of bath/bedrooms drive the property value? And if so, is there one more weighted against the other?

- Is there a difference of the average of property values by county? And if so, what is the difference?


### Follow up Questions:

- Can we get data on the population income average per county?

- Can we get the data on school ratings (greatschools.com) and see if this might be a main driver as well?

- Can we gather predictions and data based on other location factors (ie parks, hiking, waterfronts, nearby churches, grocery stores, malls, etc)





In [19]:
#will insert and collect needed imports to repo along the workflow, down below:

In [20]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import os
from env import get_db_url
import matplotlib as plt
import seaborn as sns
import numpy as np
import acquire

### Acquistion: 
INITIAL ACQUISITION:
We will be calling in the zillow dataset from the MySQL database.
This dataset will only with include taxvaluedollarcnt, bathroom count, bedroom count, calculatedfinishedsquarefeet, transactiondate and fips.

This dataset will also only take in transaction accounts from year 2017. 

In [21]:
# Using MySQL, we will Find all of the "Single Family Residential" properties and transactions from 2017:
df = pd.read_sql("SELECT bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, transactiondate, fips FROM properties_2017 JOIN predictions_2017 AS pe USING (parcelid) JOIN propertylandusetype USING (propertylandusetypeid) WHERE propertylandusetype.propertylandusedesc = 'Single Family Residential' AND pe.transactiondate LIKE '2017%%';", get_db_url("zillow"))
df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,transactiondate,fips
0,4.0,2.0,1242.0,590000.0,2017-01-13,6037.0
1,2.0,1.0,864.0,355000.0,2017-01-13,6037.0
2,3.0,2.0,1550.0,2200000.0,2017-01-13,6037.0
3,3.0,2.0,1149.0,185716.0,2017-01-13,6037.0
4,3.0,2.0,1508.0,419389.0,2017-01-13,6037.0


### Acquistion Summary: 
Getting to know the data in dtypes, and any missed nulls from prep


In [12]:
#noting that transactiondate is the only non-numeric column.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52441 entries, 0 to 52440
Data columns (total 6 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   bedroomcnt                    52441 non-null  float64
 1   bathroomcnt                   52441 non-null  float64
 2   calculatedfinishedsquarefeet  52359 non-null  float64
 3   taxvaluedollarcnt             52440 non-null  float64
 4   transactiondate               52441 non-null  object 
 5   fips                          52441 non-null  float64
dtypes: float64(5), object(1)
memory usage: 2.4+ MB


In [13]:
#Let's check for any nulls
df.isnull().sum()

bedroomcnt                       0
bathroomcnt                      0
calculatedfinishedsquarefeet    82
taxvaluedollarcnt                1
transactiondate                  0
fips                             0
dtype: int64

In [14]:
#If these nulls are dropped, will it effect the dataset?
round(df.dropna().shape[0] / df.shape[0], 4)

0.9984

In [18]:
#99% of our data still left? Great! Let's drop them!
df = df.dropna()
df.isnull().sum()

bedroomcnt                      0
bathroomcnt                     0
calculatedfinishedsquarefeet    0
taxvaluedollarcnt               0
transactiondate                 0
fips                            0
dtype: int64

In [22]:
#looking at the counts for each column
for column in df.columns:
    print(column)
    print(df[column].value_counts())
    print("-------------------------------")

bedroomcnt
3.0     23359
4.0     15240
2.0      8340
5.0      3973
6.0       635
1.0       612
0.0       137
7.0       106
8.0        24
9.0         8
12.0        3
10.0        2
14.0        1
11.0        1
Name: bedroomcnt, dtype: int64
-------------------------------
bathroomcnt
2.0     21893
3.0     10673
1.0      9568
2.5      3934
4.0      2227
3.5       918
1.5       841
5.0       803
4.5       687
6.0       322
5.5       224
0.0       121
7.0        88
8.0        53
6.5        47
7.5        16
9.0        13
10.0        5
11.0        3
8.5         3
18.0        1
13.0        1
Name: bathroomcnt, dtype: int64
-------------------------------
calculatedfinishedsquarefeet
1120.0    128
1200.0    119
1080.0    102
936.0      86
1040.0     86
         ... 
6405.0      1
8531.0      1
4533.0      1
5139.0      1
5124.0      1
Name: calculatedfinishedsquarefeet, Length: 4723, dtype: int64
-------------------------------
taxvaluedollarcnt
455000.0    34
600000.0    30
550000.0    24
50000

### Prep-Scale:

### Exploration and Visualization:

In [ ]:
plt.figure(figsize=(10,8))
sns.scatterplot(data=df, x="Bedroom_Count", y="Assessed_Value")

### Exploration and Test:

In [ ]:
plt.figure(figsize=(10,8))
sns.scatterplot(data=df, x="Bedroom_Count", y="square_feet")